# Importing Excel Inventory to bw

- based on import-xlsx from Sabrina


In [1]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from bw2data import Database

19:46:55+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bd.projects.set_current("paw_lca")

In [3]:
### Import LCI from Excel into Brightway
########################################################

print(bd.projects)

# Define variables first
projectName = "paw_lca"
database = "paw_db" # check
ei_match = "ecoinvent-3.11-cutoff"  # the EI version in the project  & file
biosphere_match = "ecoinvent-3.11-biosphere"  # the biosphere version in the project  & file
path = "LCI_ Inventories.xlsx"
# r"C:\Users\Asus\OneDrive - Universidade de Santiago de Compostela\07_Papers\Harmonization study\harmonization inventories_importer_db_test.xlsx"
##########################################################

Brightway2 projects manager with 5 objects:
	Bicycle example
	bw25-tuto
	default
	ecoinvent-3.11-cutoff-bw25_premise
	paw_lca
Use `projects.report()` to get a report on all projects.


In [4]:

# Ensure the Brightway project is set
if projectName in bd.projects:
    bd.projects.set_current(projectName)
    print(f"Project {projectName} is set as current project.")
else:
    print(f"Project {projectName} does not exist.")
    exit()
print("Available databases in the project:", list(bd.databases))
# check databases in the project
if database in bd.databases:
    print(f"Database {database} is already in the project. Will add new data to it.")
    user_input = input("Do you want to continue? (y/n): ").strip().lower()
    if user_input == "n":
        exit()
else:
    user_input = (
        input(f"Do you want to import the database {database}? (y/n): ").strip().lower()
    )
    if user_input == "y":
        print(f"Database {database} will be imported.")
    else:
        print("Process stopped by user.")
        exit()

########################################################
# Create the importer
harmonization_db = bi.ExcelImporter(path)

# apply strategies for matching
harmonization_db.apply_strategies()
# match database to itself
harmonization_db.match_database(fields=["name", "unit", "location"])
# match database to ecoinvent
harmonization_db.match_database(
    ei_match, fields=["name", "location", "reference product"]
)

# match database to biopshere flows
harmonization_db.match_database(biosphere_match, fields=["name", "categories"])
# harmonization_db.match_database("pharma-biosphere", fields=["name", "categories", "location"])

# Optionally, you can inspect unmatched exchanges or activities
print(harmonization_db.statistics())

if harmonization_db.all_linked:
    print("All flows are linked. Database will be written.")
    harmonization_db.write_database()
    print("New database created: ", list(bd.databases))

    # Check how many activities were imported
    db = bd.Database(database)
    activity_count = len(list(db))
    print(f"Number of activities imported: {activity_count}")

    # List all activities
    print("Activities in the database:")
    for i, act in enumerate(db, 1):
        print(f"{i}. {act['name']} ({act['location']})")

else:
    print("Some flows are not linked.")
    print("Unlinked flows:")
    for flow in harmonization_db.unlinked:
        print(flow)
    exit()


# Now your database is available in brightway

Project paw_lca is set as current project.
Available databases in the project: ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2045 2025-11-12', 'paw_db']
Database paw_db is already in the project. Will add new data to it.


Extracted 1 worksheets in 0.03 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 0.51 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Graph statistics for `paw_db` importer:
15 g

100%|██████████| 15/15 [00:00<00:00, 809.35it/s]


19:47:06+0100 [info     ] Vacuuming database            
Created database: paw_db
New database created:  ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2045 2025-11-12', 'paw_db']
Number of activities imported: 15
Activities in the database:
1. BPA to water (RER)
2. Ozonation (DE)
3. secondary treatment (Europe without Switzerland)
4. Electricity 102 (RER)
5. transport of fertilizer to soil (RER)
6. Activated Carbon Technology (DE)
7. GAC 230 (RER)
8. Electricity 230 (RER)
9. Chemicals 102 (RER)
10. Chemicals 230 (RER)
11. sludge incineration (Europe without Switzerland)
12. transport of sludge to incineration (RER)
13. BPA to soil (RER)
14. Emissions 230 (RE

In [5]:
# check metadata
bd.Database(database).metadata

{'format': 'Excel spreadsheet',
 'depends': ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff'],
 'backend': 'sqlite',
 'number': 15,
 'modified': '2025-11-13T19:47:06.739615',
 'geocollections': ['world'],
 'processed': '2025-11-13T19:49:19.143839',
 'dirty': False,
 'searchable': True}

In [6]:
'paw_db' in bd.databases

True

In [7]:
bd.databases

Databases dictionary with 9 object(s):
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-cutoff
	ei_cutoff_3.11_remind-eu_SSP2-NPi_2035 2025-11-12
	ei_cutoff_3.11_remind-eu_SSP2-NPi_2045 2025-11-12
	ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2035 2025-11-12
	ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2045 2025-11-12
	ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2035 2025-11-12
	ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2045 2025-11-12
	paw_db